# Read Necessary Files & Pre-processing

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
import pickle
import ast
import bitermplus as btm
from sklearn.feature_extraction import text

In [2]:
# whole_india_df = pd.read_csv("Objective & Subjective Classification/india-v1-preprocessed-overall.csv")
whole_usa_df = pd.read_csv("demo files/usa-v2-preprocessed-overall.csv")
whole_usa_df.drop(69103, axis=0, inplace=True)
filtered_usa_df = pd.read_csv('demo files/sentiment_labelled_usa_NB.csv')

In [3]:
#For obj & sub
bow_transform = CountVectorizer(tokenizer=lambda doc: doc, ngram_range=[3,3], lowercase=False) 
whole_data_bow = bow_transform.fit_transform(whole_usa_df["new_text_after_translation"])
tfidf_transform = text.TfidfTransformer(norm=None)

In [4]:
#For topic modelling
texts = filtered_usa_df['words_processed_noun']

new_text_noun = []

for row in texts:
    new_row = ast.literal_eval(row)
    new_row = " ".join(new_row)
    new_text_noun.append(new_row)
    
filtered_usa_df['new_text_noun'] = new_text_noun
texts = filtered_usa_df['new_text_noun']

stop_words = ['vaccine','people','couid', 'lot', 'thing', 'amp', 'day', 'week', 'time', 'year', 'vaccination',
             'month', 'number', 'part', 'hour', 'shit', 'person', 'go', 'pfizer','room', 'man', 'word', 'other', 
              'point', 'today', 'way', 'yesterday', 'lot', 'one', 'need', 'love', 'covidvaccine', 'use', 'bit',
             'idiot', 'thank', 'shot', 'tomorrow', 'dose', 'mask', 'life']

def get_dominant_df(p_zd):
    scores = [[]]
    
    for score in p_zd[0]:
        scores[0].append(score)
        
    topicnames = ["Children", "Vaccination Appointment", "Sports/Events", "Side Effects/Symptoms", "Undetermined", "Covid Updates", "Healthcare", 
             "Social Unrest", "Economic/Politics", "Covid Research"]
    docnames = ["Tweet"]
    df = pd.DataFrame(np.round(scores, 5), columns=topicnames, index=docnames)
    
    if np.amax(df.values) > 0.3:
        dominant_topic = topicnames[np.argmax(df.values, axis=1)[0]]
        df['dominant_topic'] = dominant_topic
    else:
        df['dominant_topic'] = "Unclassified"
    
    return df

In [5]:
#For topic modelling - GSDMM
def get_dominant_df_gsdmm(a_list):
    scores = [[]]
    
    for score in a_list:
        scores[0].append(score)
        
    topicnames = ["Covid Cases Updates", "Children", "Science/Medicine", "Social Unrest", "Side Effects/Symptoms", "Vaccination Appointment", "Healthcare",
                  "Sports/Events", "Economic/Political Impact"]
    docnames = ["Tweet"]
    df = pd.DataFrame(np.round(scores, 5), columns=topicnames, index=docnames)
    dominant_topic = topicnames[np.argmax(df.values, axis=1)[0]]
    df['dominant_topic'] = dominant_topic
    
    return df

In [6]:
#For sentiment classification
token = RegexpTokenizer(r'[a-zA-Z0-9]+')
cv = CountVectorizer(stop_words='english',ngram_range = (1,1),tokenizer = token.tokenize)
whole_data = cv.fit_transform(filtered_usa_df['new_text_after_translation'].values.astype('U'))

def get_sentiment(value):
    if value == 0:
        return "NEGATIVE"
    elif value == "1":
        return "NEUTRAL"
    else:
        return "POSITIVE"

In [49]:
tweet = ["sorry, i had vaccination pfizer had covid months after. currently, i am sick with negative test. i have been sick since last thursday. runny nose, sore throat, fever, &amp; problems breathing. friend vacccinated had covid prior vacation &amp; had it again too."]

# Objective & Subjective Classification

In [50]:
model_saved = open("pickles/usa/usa_subj.pickle", "rb") #binary read
model = pickle.load(model_saved)
model_saved.close()

In [51]:
tweet_transformed = bow_transform.transform(tweet)
tweet_transformed = tfidf_transform.fit_transform(tweet_transformed)

In [64]:
predicted_class = model.predict(tweet_transformed)
print("The predicted class is: " + str(predicted_class[0]).upper())

# Topic Modelling

## Biterm

In [53]:
model_saved = open("pickles/usa/usa_biterm.pickle", "rb") #binary read
model = pickle.load(model_saved)
model_saved.close()

In [63]:
tweet_noun_list = [" ".join(ast.literal_eval(filtered_usa_df[filtered_usa_df["new_text_after_translation"] == tweet[0]]["words_processed_noun"].iloc[0]))]
tweet_noun_list

In [62]:
X, vocabulary, vocab_dict = btm.get_words_freqs(texts, stop_words=stop_words)
new_docs_vec = btm.get_vectorized_docs(tweet_noun_list, vocabulary)
p_zd = model.transform(new_docs_vec)
dominant_df = get_dominant_df(p_zd)
dominant_df

In [61]:
print("The dominant topic is: " + str(dominant_df["dominant_topic"][0]))

## GSDMM

In [19]:
# model_saved = open("Topic Modelling/GSDMM/usa_gsdmm.model", "rb") #binary read
# model = pickle.load(model_saved)
# model_saved.close()

In [ ]:
# tweet_nouns_list = ast.literal_eval(filtered_india_df[filtered_india_df["translated"] == tweet[0]]["words_processed_noun"].iloc[0])
# tweet_nouns_list

In [ ]:
# dominant_df = get_dominant_df_gsdmm(model.score(tweet_nouns))
# dominant_df

In [ ]:
# print("The dominant topic is: " + str(dominant_df["dominant_topic"][0]))

# Sentiment Analysis

In [57]:
model_saved = open("pickles/usa/usa_sentiment.model", "rb") #binary read
model = pickle.load(model_saved)
model_saved.close()

In [58]:
tweet_transformed = cv.transform(tweet)

In [60]:
predicted_class = model.predict(tweet_transformed)
sentiment = get_sentiment(predicted_class[0])
print("The predicted class is: " + sentiment)